## config

In [13]:
import os
import glob
import re
import pandas as pd

In [1]:
## config the path where multiple files contained.
## NEED modify every time it runs !!!

path = "/home/jovyan/work/02.ResDev/13.Immuo_Reg_Target_epitope_mouse/mouse_2023-09-13-11"
homobed_or_blasp = "homoblastp"
'''
select list is "homobed","homoblastp","bedblastp"
"homobed" means homobed file as input only
"homoblastp" means homoblastp file as input only
"bedblastp" means homobed and homoblastp files both as input
'''
identity = 65

In [2]:
## config input files under the given path.
## No NEED modify every time it runs !!!

path_mhcI = path + "/02.protein_antigen_prediction/parsed_res_MHCI"
path_mhcII = path + "/02.protein_antigen_prediction/parsed_res_MHCII"
APres_FIONA = path + "/02.protein_antigen_prediction/peptide.antigen_presentation.FIONA.seq"
trans_memb = path + "/04.TransMembrane.DeepTMHMM/TransMembrane.bed"
enzymedigest = path + "/05.EnzymeDigest/protein.merge.enzymedigest"

if homobed_or_blasp=="homobed":
    homo_bed = path + "/03.homologous/homo_peptide.bed"
elif homobed_or_blasp== "homoblastp":
    homo_blastp = path + "/03.homologous/peptide_netMHC.blastp"
elif homobed_or_blasp== "bedblastp":
    homo_bed = path + "/03.homologous/homo_peptide.bed"
    homo_blastp = path + "/03.homologous/peptide_netMHC.blastp"

outpath = path + "/06.Deliverables/EpitopePresent"
outpath1 = path + "/06.Deliverables/EpitopePresent/MHCI"
outpath2 = path + "/06.Deliverables/EpitopePresent/MHCII"
try:
    os.mkdir(outpath)
except:
    pass
try:
    os.mkdir(outpath1)
except:
    pass
try:
    os.mkdir(outpath2)
except:
    pass

#### Tools

In [3]:
import difflib
def get_overlap(s1, s2):
    s = difflib.SequenceMatcher(None, s1, s2)
    pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
    return(size,s1[pos_a:pos_a+size])
s1 = "CGATTCCAGGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC"
s2 = "GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTCGTCCAGACCCCTAGC"

print(get_overlap(s1, s2)) # GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC

(39, 'GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC')


In [5]:
def get_homodict_bed(Homo_parsDict,seqid,pos1,pos2):
    checklist = [homo_f==0 for homo_f in Homo_parsDict[seqid][pos1:pos2]]
    if  sum(checklist)/(pos2-pos1) >= 0.05:
        HomoEx = "N"
        Homo_flag = ""
    else:
        HomoEx = "Y"
        flagtemp_list = []
        for flag in Homo_parsDict[seqid][pos1:pos2]:
            if flag !=0:
                flagtemp_list += flag
        Homo_flag = ";".join(list(dict.fromkeys(flagtemp_list)))
    return(HomoEx,Homo_flag)

def get_homodict_blastp(Homoblastp_parsDict,seqid,peptide):
    try:
        homolist = Homoblastp_parsDict[seqid][peptide]
        if len(homolist) == 0:
            Homo_flag = ""
            HomoEx = "N"
        else:
            Homo_flag = ';'.join(homolist)
            HomoEx = "Y"
    except:
        Homo_flag = ""
        HomoEx = "N"
    return(HomoEx,Homo_flag)
    

## Step1: Parser the TransMemb.bed

In [6]:
## Parser the TransMemb.bed
TM_parsDict={}
flagDict={"o":"outside","T":"TMhelix","i":"inside","s":"signal"}
'''
    parser the tranmembe.bed file
    TM_flag=outside,TMhelix,inside
    outside:o
    TMhelix:t
    inside:i
'''
with open(trans_memb,"r") as tmf:
    for line in tmf.readlines():
        if line.startswith("sp|"):
            #seqid = line.split("\t")[0].split("|")[1]
            seqid = line.split("\t")[0].split("|")[1].split("_")[0]
            if seqid not in TM_parsDict.keys():
                TM_parsDict[seqid] = ""
            pos1 = int(line.split("\t")[1])
            pos2 = int(line.split("\t")[2])
            TM_flag = line.split("\t")[3][0]
            TM_parsDict[seqid] += TM_flag*(1+pos2-pos1)

In [7]:
print(TM_parsDict)

{'P09581': 'sssssssssssssssssssooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooTTTTTTTTTTTTTTTTTTTTiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii', 'Q99NH8'

## Step2: Parser the Homologous_peptide.bed

In [8]:
## Parser the Homologous_peptide.bed
if (homobed_or_blasp == "homobed") or (homobed_or_blasp == "bedblastp"):
    Homobed_parsDict={}
    with open(homo_bed,"r") as hmf:
        for line in hmf.readlines():
            if line.startswith("sp|"):
                #print(line)
                seqid = line.split("\t")[0].split("|")[1]
                if seqid not in Homobed_parsDict.keys():
                    length = len(TM_parsDict[seqid])
                    Homobed_parsDict[seqid] = [0]*length
                pos1 = int(line.split("\t")[1])-1
                pos2 = int(line.split("\t")[2])-1
                homoid = line.split("\t")[3]
                for pos in range(pos1,pos2):
                    #print(pos)
                    if Homobed_parsDict[seqid][pos] == 0:
                        Homobed_parsDict[seqid][pos] = [homoid]
                    else:
                        Homobed_parsDict[seqid][pos].append(homoid)

In [18]:
## Parser the peptide_netMHC.blastp
if (homobed_or_blasp == "homoblastp") or (homobed_or_blasp == "bedblastp"):
    Homoblastp_parsDict={}
    headname= ["qseqid","sseqid","pident","length","mismatch","gapopen","qstart","qend","sstart","send","evalue","bitscore"]
    blastdf = pd.read_csv(homo_blastp,sep="\t",names=headname)
    blastdf[["protein","peptide"]] = blastdf.qseqid.str.split(":",expand=True)
    #print(blastdf.head(10))
    blastdf["proteinUnipID"] = blastdf.protein.str.split("_").str[1]
    #print(blastdf.proteinUnipID.unique().tolist())
    proteinlist = blastdf.proteinUnipID.unique().tolist()
    selfUniproList = []
    for proteinid in proteinlist:
        # calculate the self 
        qblastdf = blastdf[blastdf["proteinUnipID"]==proteinid]
        homoRefseries = qblastdf.sseqid.value_counts()
        # calculate the query number (or bin numbers)
        querynum = len(qblastdf.qseqid.unique())
        # find those reference id which is the query ploymer 
        selfUniproID = homoRefseries[homoRefseries>querynum*0.9].index.tolist()
        selfUniproList += selfUniproID
        print(selfUniproID)
    print(selfUniproList)
 
    # remove those aligned lines which reference is it's self
    noSelfBlastdf = blastdf[~blastdf.sseqid.isin(selfUniproList)]
    # select those aligned lines which pident bigger than cutoff
    noSelfHomodf = noSelfBlastdf[noSelfBlastdf.pident > identity] # identity=65
    
    noSelfHomodf2 = noSelfHomodf[["proteinUnipID","peptide","sseqid"]]
    #print(noSelfHomodf2.head(20))
    for proteinid in proteinlist:
        print(proteinid)
        noSelfHomodf2_p = noSelfHomodf2[noSelfHomodf2["proteinUnipID"]==proteinid]
        peptidelist = noSelfHomodf2_p.peptide.unique().tolist()
        peptidesseqiddict = {}
        for peptide in peptidelist:
            sseqidlist = noSelfHomodf2_p[noSelfHomodf2_p["peptide"]==peptide].sseqid.tolist()
            peptidesseqiddict[peptide] = sseqidlist
            Homoblastp_parsDict[proteinid] = peptidesseqiddict
    #print(Homoblastp_parsDict)
  

[]
[]
['P09581']
['P10923']
['Q8VCH2']
['Q99NH8']
['P09581', 'P10923', 'Q8VCH2', 'Q99NH8']
nan
A0A087WRT7
P09581
P10923
Q8VCH2
Q99NH8


In [11]:
#print(Homoblastp_parsDict["Q96A99"])

## Step3: Parser the enzymedigest file

In [9]:
## Parser the enzymedigest file
##适用于TAA的分析
EnzDigeDict={}
with open(enzymedigest,"r") as enzymef:
    for line in enzymef.readlines():
        refseq = re.split(',"\[',line)[0].split(",")[-1]
        refid = re.split(',"\[',line)[0].split("|")[1].split("_")[0]
        seqlist = re.split(',"\[',line)[1].strip().replace(']"','').replace("'","").split(",")
        EnzDigeDict[refid]=seqlist
#print(EnzDigeDict)

In [10]:
print(EnzDigeDict)

{}


## Step4: Parse the Antigen Presentation of FIONA

In [11]:
## Parser the AP result of FIONA
AP_Fiona = {}
with open(APres_FIONA,"r") as APresFiona:
    for line in APresFiona.readlines():
        #print(line)
        line = line.replace("|","_").rstrip()
        #print(line)
        seqid = line.split("_")[1]
        peptide = line.split(",")[1]
        genetype = line.split(",")[2].replace("-","")
        Apres = line.split(",")[4]
        
        if seqid not in AP_Fiona.keys():
            AP_Fiona[seqid] = {}
            AP_Fiona[seqid][":".join([peptide,genetype])] = Apres
        else:
            AP_Fiona[seqid][":".join([peptide,genetype])] = Apres


FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/work/02.ResDev/13.Immuo_Reg_Target_epitope_mouse/mouse_2023-09-13-11/02.protein_antigen_prediction/peptide.antigen_presentation.FIONA.seq'

In [15]:
print(AP_Fiona.keys())

dict_keys(['TYRP2', 'TYRP1', 'PMEL'])


## Step5:

In [28]:
## Parser the netMHC.csv
mergeoutfile =  open(outpath+"/MHCI_Epitopeonly_SB_WB.csv","w")
filenum = 0
for file in glob.glob(path_mhcI + "/*.netMHC.csv"):
    filenum +=1
    filename= os.path.basename(file)
    print(filename)
    outfile = open(outpath1+"/"+filename,"w")
    with open(file,"r") as eptf:
        for line in eptf.readlines():
            #print(line)
            itemlist = line.rstrip().split(",")
            """
            itemlist is the line content of netMHC result
            """
            if line.startswith("Pos"):
                appendlist = ["AP_result_of_FIONA","TransMemb","InCutmerRate","InCutmerRegion","HomoExsit","HomoId"]
                outfile.write(",".join(itemlist+appendlist)+"\n")
                if filenum == 1:
                    mergeoutfile.write(",".join(itemlist+appendlist)+"\n")
            else:
                #print(itemlist[0])
                pos1 = int(itemlist[0])-1
                peptide = itemlist[2]
                pos2 = pos1 + len(peptide)
                #print(itemlist)
                seqid = itemlist[10].split("_")[1].split("_")[0]
                midpos = int((pos1+pos2)/2)
                genetype =  itemlist[1].replace("-","")
                
                #print(seqid)
                
                ## get AP result of FIONA
                #APres_Fiona = AP_Fiona[seqid][":".join([peptide,genetype])]
                APres_Fiona = "-"
                
                ## get Transmembrane info
                TM_f = TM_parsDict[seqid][midpos]
                TM_flag = flagDict[TM_f]
                
                ## get enzymedigest info
                size0 = 0
                overseq0 = ""
                '''
                for seq in EnzDigeDict[seqid]:
                    size,overseq = get_overlap(peptide,seq)
                    if size > size0:
                        size0 = size
                        overseq0 = overseq
                Cut_flag = '"'+str(size0)+"//"+str(len(peptide))+'"'
                Cut_seq = overseq0
                '''
                Cut_flag = '-'
                Cut_seq = '-'
                ## get homolgous info
                if (homobed_or_blasp == "homobed") or (homobed_or_blasp == "bedblastp"):
                    HomoEx,Homo_flag = get_homodict_bed(Homo_parsDict,seqid,pos1,pos2)
                elif (homobed_or_blasp == "homoblastp") or (homobed_or_blasp == "bedblastp"):
                    HomoEx,Homo_flag = get_homodict_blastp(Homoblastp_parsDict,seqid,peptide)
                    
                ##  output the results with filtering the "SB" lines
                if (itemlist[-1] == "SB") or (itemlist[-1] == "WB"):
                    appendlist = [APres_Fiona,TM_flag,Cut_flag,Cut_seq,HomoEx,Homo_flag]
                    mergeoutfile.write(",".join(itemlist+appendlist)+"\n")
                else:
                    appendlist = ["",APres_Fiona,TM_flag,"","",HomoEx,Homo_flag]
                outfile.write(",".join(itemlist+appendlist)+"\n")
    outfile.close()

sp_A0A087WRT7_A_MHCI_H-2-Db.netMHC.csv
sp_Q99NH8_TREM2_MHCI_H-2-Kb.netMHC.csv
sp_P09581_CSF1R_MHCI_H-2-Db.netMHC.csv
sp_Q8VCH2_CLM5__MHCI_H-2-Db.netMHC.csv
sp_Q8VCH2_CLM5__MHCI_H-2-Kb.netMHC.csv
sp_A0A087WRT7_A_MHCI_H-2-Kb.netMHC.csv
sp_Q99NH8_TREM2_MHCI_H-2-Db.netMHC.csv
sp_P09581_CSF1R_MHCI_H-2-Kb.netMHC.csv
sp_P10923_OSTP__MHCI_H-2-Kb.netMHC.csv
sp_P10923_OSTP__MHCI_H-2-Db.netMHC.csv


In [17]:
print(seqid)

TYRP1


In [30]:
## Parser the netMHC.csv
mergeoutfile2 =  open(outpath+"/MHCII_Epitope_only_SB_WB.csv","w")
filenum = 0
for file in glob.glob(path_mhcII + "/*.netMHC.csv"):
    filenum += 1
    filename= os.path.basename(file)
    print(filename)
    outfile = open(outpath2+"/"+filename,"w")
    with open(file,"r") as eptf:
        for line in eptf.readlines():
            itemlist = line.rstrip().split(",")
            #print(itemlist)
            if line.startswith("Pos"):
                appendlist = ["AP_result_of_FIONA","TransMemb","InCutmerRate","InCutmerRegion","HomoExsit","HomoId"]
                outfile.write(",".join(itemlist+appendlist)+"\n")
                if filenum == 1:
                    mergeoutfile2.write(",".join(itemlist+appendlist)+"\n")
            else:
                pos1 = int(itemlist[0])-1
                peptide = itemlist[2]
                pos2 = pos1 + len(peptide)
                seqid = itemlist[6].split("|")[1].split("_")[0]
                midpos = int((pos1+pos2)/2)
                genetype =  itemlist[1].replace("-","")
                
                ## get AP result of FIONA
                #APres_Fiona = AP_Fiona[seqid][":".join([peptide,genetype])]              
                APres_Fiona = "-"              
                    
                ## get Transmembrane info
                TM_f = TM_parsDict[seqid][midpos]
                TM_flag = flagDict[TM_f]
                
                ## get enzymedigest info
                size0 = 0
                overseq0 = ""
                '''
                for seq in EnzDigeDict[seqid]:
                    size,overseq = get_overlap(peptide,seq)
                    if size > size0:
                        size0 = size
                        overseq0 = overseq
                Cut_flag = '"'+str(size0)+"//"+str(len(peptide))+'"'
                Cut_seq = overseq0
                '''
                Cut_flag = '-'
                Cut_seq = '-'
                ## get homolgous info
                if (homobed_or_blasp == "homobed") or (homobed_or_blasp == "bedblastp"):
                    HomoEx,Homo_flag = get_homodict_bed(Homo_parsDict,seqid,pos1,pos2)
                elif (homobed_or_blasp == "homoblastp") or (homobed_or_blasp == "bedblastp"):
                    HomoEx,Homo_flag = get_homodict_blastp(Homoblastp_parsDict,seqid,peptide)
                        
                ##  output the results with filtering the "SB" lines
                if (itemlist[-1] == "SB") or (itemlist[-1] == "WB"):
                    appendlist = [APres_Fiona,TM_flag,Cut_flag,Cut_seq,HomoEx,Homo_flag]
                    mergeoutfile2.write(",".join(itemlist+appendlist)+"\n")
                else:
                    appendlist = ["",APres_Fiona,TM_flag,"","",HomoEx,Homo_flag]
                outfile.write(",".join(itemlist+appendlist)+"\n")
    outfile.close()

sp_Q99NH8_TREM2_MHCII_H-2-IAb.netMHC.csv
sp_P09581_CSF1R_MHCII_H-2-IAb.netMHC.csv
sp_Q8VCH2_CLM5__MHCII_H-2-IAb.netMHC.csv
sp_P10923_OSTP__MHCII_H-2-IAb.netMHC.csv
sp_A0A087WRT7_A_MHCII_H-2-IAb.netMHC.csv


In [31]:
df_SB_WB = pd.read_csv(outpath+"/MHCI_Epitopeonly_SB_WB.csv")
proteinlist = list(df_SB_WB["Identity"].unique())
print(proteinlist)
for seqid in proteinlist:
    try:
        os.mkdir(outpath+"/"+seqid)
    except:
        pass
    seqid_df = df_SB_WB[df_SB_WB["Identity"].str.contains(seqid)]
    MHClist= pd.Series(seqid_df["MHC"].unique())
    MHClist2 = MHClist.str.replace('\*', r'\\*', regex=True)
    print(MHClist2)
    #MHClist2 = MHClist
    
    for MHCid,MHCid2 in zip(MHClist,MHClist2):
        seqid_mhcid_df = seqid_df[seqid_df["MHC"].str.contains(MHCid2)]
        seqid_mhcid_df.to_csv(outpath+"/"+seqid+"/"+seqid+"_MHCI_"+MHCid+".netMHC.csv", index=False)

['sp_A0A087WRT7_A', 'sp_Q99NH8_TREM2', 'sp_P09581_CSF1R', 'sp_Q8VCH2_CLM5_', 'sp_P10923_OSTP_']
0    H-2-Db
1    H-2-Kb
dtype: object
0    H-2-Kb
1    H-2-Db
dtype: object
0    H-2-Db
1    H-2-Kb
dtype: object
0    H-2-Db
1    H-2-Kb
dtype: object
0    H-2-Kb
1    H-2-Db
dtype: object


In [33]:
df_SB_WB = pd.read_csv(outpath+"/MHCII_Epitope_only_SB_WB.csv")
df_SB_WB["Identity"] = df_SB_WB["Identity"].str.replace("|","_")
proteinlist = list(df_SB_WB["Identity"].unique())
for seqid in proteinlist:
    try:
        os.mkdir(outpath+"/"+seqid2)
    except:
        pass
    seqid_df = df_SB_WB[df_SB_WB["Identity"].str.contains(seqid)]
    MHClist= (seqid_df["MHC"].unique())
    for MHCid in MHClist:
        seqid_mhcid_df = seqid_df[seqid_df["MHC"].str.contains(MHCid)]
        seqid_mhcid_df.to_csv(outpath+"/"+seqid+"/"+seqid+"_MHCII_"+MHCid+".netMHC.csv", index=False)